In [7]:
import pandas as pd
import tensorflow as tf

# Note: Classification is an example of supervised Learning 

# Defining constants we'd be needing for the Datasets
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

# Keras used here to input the Datasets in to panda dataframe
train_path = tf.keras.utils.get_file("iris_training.csv", './data/iris_training.csv')
test_path = tf.keras.utils.get_file("iris_test.csv", './data/iris_test.csv')

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

train_y = train.pop('Species')
test_y = test.pop('Species')

# displaying the first few data
train.head()


,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [8]:
# Training the Model
def input_fn(features, labels, training=True, batch_size=256): 
    #convert the input into Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # shuffle the data when in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [9]:
# work on feature columns
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
# Building the Model  -used Deep Neural Network Classifier here instead of linear classification
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    # introducing hidden layers -- still yet to grasp this concept
    hidden_units = [30, 10],
    # having the models choose between three classes
    n_classes =3
)


In [14]:
# train the model using the classfier
classifier.train(
    input_fn= lambda: input_fn(train, train_y, training=True),
    steps=5000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmp1xdk0kdy/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 10000...
INFO:tensorflow:Saving checkpoints for 10000 into /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmp1xdk0kdy/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...
INFO:tensorflow:loss = 0.38204274, step = 10000
INFO:tensorflow:global_step/sec: 122.197
INFO:tensorflow:loss = 0.3732824, step = 10100 (0.821 sec)
INFO:tensorflow:global_step/sec: 118.94
INFO:tensorflow:loss = 0.37140498, step = 10200 (0.842 sec)
INFO:tensorflow:global_step/sec: 120.751
INFO:tensorflow:loss = 0.3747006, step = 10300 (0.827 sec)
INFO:tensorflow

In [17]:
# Testing the Accuracy of the alg.
eval_result = classifier.evaluate(input_fn= lambda: input_fn(test, test_y, training=False))
print('\n Test Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-03-18T19:09:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmp1xdk0kdy/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.83700s
INFO:tensorflow:Finished evaluation at 2023-03-18-19:09:51
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.93333334, average_loss = 0.33740938, global_step = 15000, loss = 0.33740938
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmp1xdk0kdy/model.ckpt-15000

 Test Accuracy: 0.933



In [19]:
# A Predictive Model : Users placed in the numerical feats of the Flower 
# and it predicts what class of flower it is

def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type Numeric Values as prompted: ")
for feature in features: 
    valid = True
    while valid:
        val  = input(feature + ": ")
        if not val.isdigit(): valid = False
        # placing the Numerical Inputs in here
        predict[feature] = [float(val)]
        
predictions = classifier.predict(input_fn=lambda: input_fn(predict))
# placing every prediction into dictionary
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability
    ))

Please type Numeric Values as prompted: 
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmp1xdk0kdy/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-4.256638  ,  0.50203544,  5.3861055 ], dtype=float32), 'probabilities': array([6.4403270e-05, 7.5088572e-03, 9.9242675e-01], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (99.2%)
